In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from keras_preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import img_to_array

from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications import ResNet50

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
HEIGHT = 150
WIDTH=150
SEED = 40
BATCH_SIZE=32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    validation_split=0.2
    )

In [ ]:
train_ds = datagen.flow_from_dataframe(
    df,
    directory = '../input/resized-plant2021/img_sz_256',
    subset='training',
    x_col='image',
    y_col='labels',
    target_size=(HEIGHT,WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

In [ ]:
val_ds = datagen.flow_from_dataframe(
    df,
    directory = '../input/resized-plant2021/img_sz_256',
    subset='validation',
    x_col='image',
    y_col='labels',
    target_size=(HEIGHT,WIDTH),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

In [ ]:


pretrained_model = ResNet50(input_shape=(HEIGHT,WIDTH,3), include_top=False, weights='../input/resnet50weight/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

for layer in pretrained_model.layers:
    layer.trainable=False
pretrained_model.summary()

In [ ]:
last_layer = pretrained_model.get_layer('conv5_block3_out')
print('last layer of Resnet50 : output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)                  
x = Dense(12, activation='softmax')(x)


model = Model(pretrained_model.input, x) 

In [ ]:
from tensorflow import keras
import tensorflow_addons as tfa
lrschedule = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                 factor=0.05, patience=5, verbose=1)

f1 = tfa.metrics.F1Score(num_classes=12,average='macro',threshold=0.23)
earlystop=EarlyStopping(patience=4,monitor=f1,mode='max',restore_best_weights=True)
callbacks=[lrschedule,earlystop]
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(), 
              metrics= ['accuracy',f1])

In [ ]:
resNet_model=model.fit_generator(train_ds,
                        validation_data=val_ds,
                        epochs=15,
                        callbacks=callbacks)


In [ ]:
model.save('resnet50ver8.h5')

In [ ]:
# submission = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv')
# submission.head()

In [ ]:
# test_datagen = ImageDataGenerator(
#     rescale = 1/255.0
# )

# test_generator = test_datagen.flow_from_dataframe(
#     submission,
#     directory="../input/plant-pathology-2021-fgvc8/test_images",
#     x_col='image',
#     y_col=None,
#     class_mode=None,
#     color_mode="rgb",
#     target_size=(HEIGHT,WIDTH),
# )

In [ ]:
# from tensorflow_addons.metrics import F1Score
# preds = tf.keras.models.load_model('../input/resnetfan50/resnet50ver5.h5')


In [ ]:
# preds = preds.predict(test_generator)


In [ ]:
# dict_classes = train_ds.class_indices
# dict_classes

In [ ]:


# # indices = []
# # for pred in preds:
# #     temp = []
# #     for category in pred:
# #         if category>=0.23:
# #             temp.append(pred.index(category))
# #     if temp!=[]:
# #         indices.append(temp)
# #     else:
# #         temp.append(np.argmax(pred))
# #         indices.append(temp)
    
# # print(indices)

# for i in range(len(preds)):
#     preds[i] = np.argmax(preds[i])

    
# def get_key(val):
#     for key, value in dict_classes.items():
#         if val == value:
#             return key
        

# for i in range(len(preds)):
#     preds[i] = get_key(preds[i])

In [ ]:
# labels = (train_ds.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# print(labels)

# testlabels = []

# for image in indices:
#     temp = []
#     for i in image:
#         temp.append(str(labels[i]))
#     testlabels.append(' '.join(temp))

# print(testlabels)



In [ ]:
# submission['labels'] = preds
# submission.head()

In [ ]:
# thresh = 0.4
# for i in range(3):
#     submission.iloc[i, 1] = ' '.join(train_df.columns[1:][preds[i] >= thresh])
    
# submission.head() 

In [ ]:
# submission.to_csv('submission.csv', index=False)